In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
sb.set() # set the default Seaborn style for graphics

train = pd.read_csv('Hotel Reservations.csv')
train.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


# Data Cleaning:

## Error Checking:
1. Check for data errors (date invalid), remove rows. Example being 29 Feb 2018, there is no leap year in 2018.
2. Ensure that the booking has at least 1 guest (no_of_adults & no_of_children added is not 0)
3. Ensure at least one date is booked (no_of_weekend_nights, no_of_week_nights added is not 0)

## 
1. To find the date, we will combine the arrival year/month/date columns to a new datetime column, so that we can easily use datetime to further analyse.
5. Change booking_status from Not_Cancelled, Cancelled to 1, 0 respectively for further analysis


In [2]:
year = train["arrival_year"].astype(str)
month = train["arrival_month"].astype(str)
date = train["arrival_date"].astype(str)

#combine year month date columns to a new column with format as shown
train["arrival_datetime"] = year + "/" + month + "/" + date

#drop year month date columns
train.drop(['arrival_year', 'arrival_month', 'arrival_date'], axis=1, inplace=True)

#convert string to datetime format, invalid dates will be replaced with NaT (Not a Time)
train["arrival_datetime"] = pd.to_datetime(train["arrival_datetime"], format = "%Y/%m/%d",errors='coerce')


In [3]:
#drop any invalid dates (NaT)
train = train.dropna(subset=['arrival_datetime'])
train.reset_index(drop=True, inplace=True)
train.shape

(36238, 17)

Number of rows has been reduced from 36275 to 36238, thus there were 37 cases of invalid dates that was removed

In [4]:
#drop any rows where both adults and children are 0 (nobody booked)
train = train[~((train['no_of_adults'] == 0) & (train['no_of_children'] == 0))]
train.reset_index(drop=True, inplace=True)
train.shape

(36238, 17)

There was no change in rows after this, so there was at least one person per booking.

In [5]:
#drop any rows where both weekend nights and week nights are 0 (did not stay a night)
train = train[~((train['no_of_weekend_nights'] == 0) & (train['no_of_week_nights'] == 0))]
train.reset_index(drop=True, inplace=True)
train.shape

(36160, 17)

Number of rows has been reduced from 36238 to 36160, thus there were 78 cases of invalid dates that was removed

In [6]:
#exporting for EDA
train.to_csv("CleanedEDA.csv")

### Changing using encoding


In [7]:

train['booking_status'] = train['booking_status'].replace({'Not_Canceled': 1, 'Canceled': 0})


label_encoder = LabelEncoder()
train['type_of_meal_plan'] = label_encoder.fit_transform(train['type_of_meal_plan'])
train['room_type_reserved'] = label_encoder.fit_transform(train['room_type_reserved'])
train.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status,arrival_datetime
0,INN00001,2,0,1,2,0,0,0,224,Offline,0,0,0,65.00,0,1,2017-10-02
1,INN00002,2,0,2,3,3,0,0,5,Online,0,0,0,106.68,1,1,2018-11-06
2,INN00003,1,0,2,1,0,0,0,1,Online,0,0,0,60.00,0,0,2018-02-28
3,INN00004,2,0,0,2,0,0,0,211,Online,0,0,0,100.00,0,0,2018-05-20
4,INN00005,2,0,1,1,3,0,0,48,Online,0,0,0,94.50,0,0,2018-04-11


In [8]:
train.to_csv("Cleaned.csv")

Output to a new CSV file for further analysis